In [2]:
#packages
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [3]:
df = pd.read_csv('GE.csv') #data can be downloaded from here https://finance.yahoo.com/quote/GE/history?p=GE
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-07-09,6.83,6.83,6.58,6.58,6.553280,74748000
1,2020-07-10,6.54,6.72,6.53,6.69,6.662834,63423400
2,2020-07-13,6.76,6.82,6.64,6.70,6.672793,68492400
3,2020-07-14,6.66,6.89,6.57,6.88,6.852062,69509200
4,2020-07-15,7.09,7.21,7.03,7.14,7.111006,89827100


In [4]:
train_dates = pd.to_datetime(df['Date'])
train_dates

0     2020-07-09
1     2020-07-10
2     2020-07-13
3     2020-07-14
4     2020-07-15
         ...    
247   2021-07-01
248   2021-07-02
249   2021-07-06
250   2021-07-07
251   2021-07-08
Name: Date, Length: 252, dtype: datetime64[ns]

In [5]:
df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [6]:
df_for_training = df[list(df)[1:5]].astype(float)
df_for_training

,Open,High,Low,Close
0,6.83,6.83,6.58,6.58
1,6.54,6.72,6.53,6.69
2,6.76,6.82,6.64,6.70
3,6.66,6.89,6.57,6.88
4,7.09,7.21,7.03,7.14
...,...,...,...,...
247,13.58,13.63,13.36,13.48
248,13.54,13.54,13.31,13.36
249,13.28,13.32,12.82,12.92
250,12.88,13.08,12.74,12.98


In [7]:
# scaling the data for efficient computtation
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

In [8]:
#As required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features. 
#In this example, the n_features is 2. We will make timesteps = 3. 
#With this, the resultant n_samples is 5 (as the input data has 9 rows).
trainX = []
trainY = []

n_future = 1   # Number of days we want to predict into the future
n_past = 14     # Number of past days we want to use to predict the future

for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (238, 14, 4).
trainY shape == (238, 1).


In [9]:
trainX

array([[[-1.22993347, -1.27950828, -1.27330806, -1.31677884],
        [-1.33110126, -1.31765121, -1.29096079, -1.27838168],
        [-1.25435328, -1.28297582, -1.25212479, -1.27489103],
        ...,
        [-1.16365112, -1.18241718, -1.18857497, -1.21904061],
        [-1.22644492, -1.2725732 , -1.23447206, -1.27140038],
        [-1.27528455, -1.23443027, -1.23447206, -1.20856866]],

       [[-1.33110126, -1.31765121, -1.29096079, -1.27838168],
        [-1.25435328, -1.28297582, -1.25212479, -1.27489103],
        [-1.28923872, -1.25870304, -1.27683861, -1.21205931],
        ...,
        [-1.22644492, -1.2725732 , -1.23447206, -1.27140038],
        [-1.27528455, -1.23443027, -1.23447206, -1.20856866],
        [-1.17411675, -1.22056011, -1.29449134, -1.31328819]],

       [[-1.25435328, -1.28297582, -1.25212479, -1.27489103],
        [-1.28923872, -1.25870304, -1.27683861, -1.21205931],
        [-1.13923131, -1.14774178, -1.11443351, -1.12130238],
        ...,
        [-1.27528455, -1.23

In [10]:
trainY

array([[-1.17411675],
       [-1.34505544],
       [-1.43226905],
       [-1.47064304],
       [-1.48459722],
       [-1.46017741],
       [-1.4043607 ],
       [-1.41831487],
       [-1.36947525],
       [-1.21597929],
       [-1.21597929],
       [-1.31365854],
       [-1.33110126],
       [-1.28575018],
       [-1.34854398],
       [-1.35552107],
       [-1.4043607 ],
       [-1.43226905],
       [-1.39738361],
       [-1.29272727],
       [-1.3031929 ],
       [-1.33807835],
       [-1.34156689],
       [-1.34156689],
       [-1.42529196],
       [-1.44971178],
       [-1.38342943],
       [-1.36947525],
       [-1.40784924],
       [-1.44971178],
       [-1.46017741],
       [-1.50552849],
       [-1.52297121],
       [-1.46017741],
       [-1.48459722],
       [-1.23691056],
       [-1.19853656],
       [-1.29970436],
       [-1.37994088],
       [-1.42180342],
       [-1.48459722],
       [-1.4985514 ],
       [-1.43924614],
       [-1.44622323],
       [-1.47064304],
       [-1

In [11]:
#train-test split
trainx, testx = train_test_split(trainX)
trainy, testy = train_test_split(trainY)

In [12]:
print(trainx.shape,trainy.shape,testx.shape,testy.shape)

(178, 14, 4) (178, 1) (60, 14, 4) (60, 1)


In [16]:
trainx.shape[2]

4

In [17]:
#building the LSTM model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainx.shape[1], trainx.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(4))

model.compile(optimizer='adam', loss='mse')
model.summary()


# fit model
history = model.fit(trainx, trainy, epochs=500, batch_size=16, validation_split=0.1, verbose=1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 14, 64)            17664     
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 132       
Total params: 30,212
Trainable params: 30,212
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
10/10 [==============================] - 6s 179ms/step - loss: 1.0143 - val_loss: 0.9227
Epoch 2/500
10/10 [==============================] - 0s 15ms/step - loss: 1.0474 - val_loss: 0.9066
Epoch 3/500
10/10 [============================

10/10 [==============================] - 0s 16ms/step - loss: 0.9395 - val_loss: 0.9313
Epoch 74/500
10/10 [==============================] - 0s 19ms/step - loss: 0.9828 - val_loss: 0.9242
Epoch 75/500
10/10 [==============================] - 0s 15ms/step - loss: 0.8608 - val_loss: 0.9134
Epoch 76/500
10/10 [==============================] - 0s 26ms/step - loss: 1.0256 - val_loss: 0.9533
Epoch 77/500
10/10 [==============================] - 0s 19ms/step - loss: 0.9316 - val_loss: 0.9406
Epoch 78/500
10/10 [==============================] - 0s 19ms/step - loss: 0.9764 - val_loss: 0.9027
Epoch 79/500
10/10 [==============================] - 0s 17ms/step - loss: 0.9624 - val_loss: 0.9229
Epoch 80/500
10/10 [==============================] - 0s 16ms/step - loss: 0.9059 - val_loss: 0.9811
Epoch 81/500
10/10 [==============================] - 0s 21ms/step - loss: 0.9298 - val_loss: 0.8818
Epoch 82/500
10/10 [==============================] - 0s 18ms/step - loss: 0.9709 - val_loss: 0.9009
Epo

10/10 [==============================] - 0s 23ms/step - loss: 0.9478 - val_loss: 0.9361
Epoch 154/500
10/10 [==============================] - 0s 13ms/step - loss: 0.9206 - val_loss: 0.9398
Epoch 155/500
10/10 [==============================] - 0s 18ms/step - loss: 0.8993 - val_loss: 0.9933
Epoch 156/500
10/10 [==============================] - 0s 19ms/step - loss: 0.9294 - val_loss: 0.9419
Epoch 157/500
10/10 [==============================] - 0s 22ms/step - loss: 0.9302 - val_loss: 0.9525
Epoch 158/500
10/10 [==============================] - 0s 20ms/step - loss: 0.9477 - val_loss: 0.9106
Epoch 159/500
10/10 [==============================] - 0s 20ms/step - loss: 0.8434 - val_loss: 1.0196
Epoch 160/500
10/10 [==============================] - 0s 16ms/step - loss: 0.9049 - val_loss: 0.9872
Epoch 161/500
10/10 [==============================] - 0s 18ms/step - loss: 0.9143 - val_loss: 1.0200
Epoch 162/500
10/10 [==============================] - 0s 22ms/step - loss: 0.9281 - val_loss: 1

10/10 [==============================] - 0s 18ms/step - loss: 0.9086 - val_loss: 1.0809
Epoch 234/500
10/10 [==============================] - 0s 19ms/step - loss: 0.8934 - val_loss: 1.0023
Epoch 235/500
10/10 [==============================] - 0s 12ms/step - loss: 0.8800 - val_loss: 1.1060
Epoch 236/500
10/10 [==============================] - 0s 26ms/step - loss: 0.8447 - val_loss: 1.2392
Epoch 237/500
10/10 [==============================] - 0s 17ms/step - loss: 0.9081 - val_loss: 1.0623
Epoch 238/500
10/10 [==============================] - 0s 19ms/step - loss: 0.8667 - val_loss: 1.0266
Epoch 239/500
10/10 [==============================] - 0s 19ms/step - loss: 0.8303 - val_loss: 1.0663
Epoch 240/500
10/10 [==============================] - 0s 13ms/step - loss: 0.9269 - val_loss: 1.1226
Epoch 241/500
10/10 [==============================] - 0s 25ms/step - loss: 0.9368 - val_loss: 1.1394
Epoch 242/500
10/10 [==============================] - 0s 18ms/step - loss: 0.8386 - val_loss: 1

10/10 [==============================] - 0s 17ms/step - loss: 0.8243 - val_loss: 0.9537
Epoch 314/500
10/10 [==============================] - 0s 21ms/step - loss: 0.8068 - val_loss: 0.9964
Epoch 315/500
10/10 [==============================] - 0s 20ms/step - loss: 0.8996 - val_loss: 1.0221
Epoch 316/500
10/10 [==============================] - 0s 18ms/step - loss: 0.8498 - val_loss: 1.0750
Epoch 317/500
10/10 [==============================] - 0s 14ms/step - loss: 0.8351 - val_loss: 1.0389
Epoch 318/500
10/10 [==============================] - 0s 21ms/step - loss: 0.8009 - val_loss: 1.0256
Epoch 319/500
10/10 [==============================] - 0s 21ms/step - loss: 0.7968 - val_loss: 1.1003
Epoch 320/500
10/10 [==============================] - 0s 19ms/step - loss: 0.8223 - val_loss: 1.1515
Epoch 321/500
10/10 [==============================] - 0s 21ms/step - loss: 0.8420 - val_loss: 1.1163
Epoch 322/500
10/10 [==============================] - 0s 15ms/step - loss: 0.8091 - val_loss: 1

10/10 [==============================] - 0s 13ms/step - loss: 0.7633 - val_loss: 1.0821
Epoch 394/500
10/10 [==============================] - 0s 14ms/step - loss: 0.8776 - val_loss: 1.1038
Epoch 395/500
10/10 [==============================] - 0s 21ms/step - loss: 0.8599 - val_loss: 1.1047
Epoch 396/500
10/10 [==============================] - 0s 17ms/step - loss: 0.8402 - val_loss: 1.2446
Epoch 397/500
10/10 [==============================] - 0s 13ms/step - loss: 0.7627 - val_loss: 1.1892
Epoch 398/500
10/10 [==============================] - 0s 19ms/step - loss: 0.8037 - val_loss: 1.0404
Epoch 399/500
10/10 [==============================] - 0s 12ms/step - loss: 0.7539 - val_loss: 1.0124
Epoch 400/500
10/10 [==============================] - 0s 25ms/step - loss: 0.8744 - val_loss: 1.0656
Epoch 401/500
10/10 [==============================] - 0s 17ms/step - loss: 0.8322 - val_loss: 1.1444
Epoch 402/500
10/10 [==============================] - 0s 15ms/step - loss: 0.8627 - val_loss: 0

10/10 [==============================] - 0s 15ms/step - loss: 0.7313 - val_loss: 1.2651
Epoch 474/500
10/10 [==============================] - 0s 22ms/step - loss: 0.6589 - val_loss: 1.0218
Epoch 475/500
10/10 [==============================] - 0s 22ms/step - loss: 0.8082 - val_loss: 0.9643
Epoch 476/500
10/10 [==============================] - 0s 23ms/step - loss: 0.8286 - val_loss: 1.0460
Epoch 477/500
10/10 [==============================] - 0s 15ms/step - loss: 0.8043 - val_loss: 1.1354
Epoch 478/500
10/10 [==============================] - 0s 15ms/step - loss: 0.7745 - val_loss: 1.0896
Epoch 479/500
10/10 [==============================] - 0s 14ms/step - loss: 0.8249 - val_loss: 1.1586
Epoch 480/500
10/10 [==============================] - 0s 21ms/step - loss: 0.8378 - val_loss: 1.2045
Epoch 481/500
10/10 [==============================] - 0s 22ms/step - loss: 0.7865 - val_loss: 1.1365
Epoch 482/500
10/10 [==============================] - 0s 17ms/step - loss: 0.7520 - val_loss: 1

In [20]:
#testing the model
x_input = testx
temp_input=list(x_input)
lst_output=[]
i=0
while(i<10):
    
    if(len(temp_input)>14):
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        #print(x_input)
        #x_input = x_input.reshape((1, n_steps, n_features))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        scaler.inverse_transform(yhat)
        print("{} day output {}".format(i,yhat))
        temp_input.append(yhat[0][0])
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.append(yhat[0][0])
        i=i+1
    else:
        #x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.append(yhat[0][0])
        lst_output.append(yhat[0][0])
        i=i+1
    print('\n')
    

print(lst_output)
lstm = lst_output

0 day input [[[ 0.12362184  0.08323471  0.10713525  0.08995352]
  [ 0.1131562   0.15952057  0.15303234  0.1772198 ]
  [ 0.17246146  0.11444256  0.15303234  0.10391613]
  ...
  [ 0.32595743  0.31902739  0.3436818   0.34477105]
  [ 0.26316362  0.35023524  0.30837634  0.38316821]
  [ 0.40270541  0.48200174  0.43194544  0.49835969]]

 [[-1.35552107 -1.3696643  -1.35451061 -1.38659186]
  [-1.4043607  -1.43554755 -1.40393825 -1.42498902]
  [-1.43226905 -1.43554755 -1.39687716 -1.41102642]
  ...
  [-1.38342943 -1.34192399 -1.37922443 -1.40753577]
  [-1.36947525 -1.37313184 -1.38628552 -1.37262926]
  [-1.40784924 -1.44248263 -1.43218261 -1.47036748]]

 [[-1.31365854 -1.30378106 -1.26977752 -1.30979754]
  [-1.33110126 -1.31765121 -1.31214406 -1.28885363]
  [-1.28575018 -1.32805383 -1.31920516 -1.355176  ]
  ...
  [-1.34156689 -1.35232661 -1.31214406 -1.30630689]
  [-1.34156689 -1.32805383 -1.38628552 -1.40055446]
  [-1.42529196 -1.42861247 -1.42512152 -1.45291423]]

 ...

 [[ 0.99226944  0.9813

/tmp/ipykernel_8745/1560975069.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_input=np.array(temp_input[1:])


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).